In [1]:
from pipeline.backend.pipeline import PipeLine
from pipeline.component import DataTransform
from pipeline.component import HeteroSecureBoost
from pipeline.component import Intersection
from pipeline.component import Reader
from pipeline.interface import Data
from pipeline.component import Evaluation
from pipeline.interface import Model

from pipeline.utils.tools import load_job_config

In [2]:
guest=9999
host=10000
otherhost=8888
hosts=[host,otherhost]
namespace=""

guest_train_data = {"name": "breast_hetero_guest", "namespace": f"experiment{namespace}"}
host_train_data = {"name": "breast_hetero_host", "namespace": f"experiment{namespace}"}

guest_validate_data = {"name": "breast_hetero_guest", "namespace": f"experiment{namespace}"}
host_validate_data = {"name": "breast_hetero_host", "namespace": f"experiment{namespace}"}

# init pipeline
pipeline = PipeLine().set_initiator(role="guest", party_id=guest).set_roles(guest=guest, host=[host,otherhost])

# set data reader and data-io

reader_0, reader_1 = Reader(name="reader_0"), Reader(name="reader_1")

reader_0.get_party_instance(role="guest", party_id=guest).component_param(table=guest_train_data)
reader_0.get_party_instance(role="host", party_id=host).component_param(table=host_train_data)
reader_0.get_party_instance(role="host", party_id=otherhost).component_param(table=host_train_data)

reader_1.get_party_instance(role="guest", party_id=guest).component_param(table=guest_validate_data)
reader_1.get_party_instance(role="host", party_id=host).component_param(table=host_validate_data)
reader_1.get_party_instance(role="host", party_id=otherhost).component_param(table=host_train_data)


data_transform_0, data_transform_1 = DataTransform(name="data_transform_0"), DataTransform(name="data_transform_1")

data_transform_0.get_party_instance(
    role="guest", party_id=guest).component_param(
    with_label=True, output_format="dense")
data_transform_0.get_party_instance(role="host", party_id=hosts).component_param(with_label=False)
data_transform_1.get_party_instance(
    role="guest", party_id=guest).component_param(
    with_label=True, output_format="dense")
data_transform_1.get_party_instance(role="host", party_id=hosts).component_param(with_label=False)

# data intersect component
intersect_0 = Intersection(name="intersection_0")
intersect_1 = Intersection(name="intersection_1")

# secure boost component
hetero_secure_boost_0 = HeteroSecureBoost(name="hetero_secure_boost_0",
                                          num_trees=3,
                                          task_type="classification",
                                          objective_param={"objective": "cross_entropy"},
                                          encrypt_param={"method": "Paillier"},
                                          tree_param={"max_depth": 3},
                                          validation_freqs=1)

# evaluation component
evaluation_0 = Evaluation(name="evaluation_0", eval_type="binary")

pipeline.add_component(reader_0)
pipeline.add_component(reader_1)
pipeline.add_component(data_transform_0, data=Data(data=reader_0.output.data))
pipeline.add_component(
    data_transform_1, data=Data(
        data=reader_1.output.data), model=Model(
        data_transform_0.output.model))
pipeline.add_component(intersect_0, data=Data(data=data_transform_0.output.data))
pipeline.add_component(intersect_1, data=Data(data=data_transform_1.output.data))
pipeline.add_component(hetero_secure_boost_0, data=Data(train_data=intersect_0.output.data,
                                                        validate_data=intersect_1.output.data))
pipeline.add_component(evaluation_0, data=Data(data=hetero_secure_boost_0.output.data))

pipeline.compile()
pipeline.fit()

print("fitting hetero secureboost done, result:")
print(pipeline.get_component("hetero_secure_boost_0").get_summary())

2022-08-25 17:55:16.668 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:123 - Job id is 202208251755101400770

2022-08-25 17:55:16.710 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-08-25 17:55:17.250 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:00
2022-08-25 17:55:17.793 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
2022-08-25 17:55:18.334 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:01
2022-08-25 17:55:18.885 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:02
2022-08-25 17:55:19.443 | INFO     | pipeline.utils.invoker.job_submitter:monitor_job_status:144 - Job is still waiting, time elapse: 0:00:02
2022-08-25 17:55

fitting hetero secureboost done, result:
{'best_iteration': -1, 'feature_importance': {'host_10000_1': 1, 'host_10000_13': 2, 'host_10000_7': 1, 'x0': 1, 'x1': 6, 'x2': 2, 'x3': 2, 'x4': 1, 'x6': 2, 'x7': 3}, 'is_converged': False, 'loss_history': [0.4721718463760301, 0.34085371588764884, 0.25373153106142293], 'validation_metrics': {'train': {'auc': [0.9787207335764494, 0.9932482426933038, 0.9954481792717087], 'ks': [0.899172876697849, 0.9634400930183393, 0.9671264732308018]}}}
